# Библиотеки

In [ ]:
import os
from google.colab import drive
import pandas as pd
import numpy as np
import tqdm
import gc
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.linear_model import ElasticNetCV
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression

In [ ]:
!pip install tensorflow scikeras scikit-learn

# Загрузка датасета

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! unzip /content/drive/MyDrive/train.zip -d train

! unzip /content/drive/MyDrive/test.zip -d test

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: train/788639531.csv     
  inflating: train/788713777.csv     
  inflating: train/789346799.csv     
  inflating: train/789459556.csv     
  inflating: train/789627929.csv     
  inflating: train/789781713.csv     
  inflating: train/790331967.csv     
  inflating: train/791207230.csv     
  inflating: train/792193372.csv     
  inflating: train/792396274.csv     
  inflating: train/792531498.csv     
  inflating: train/79312563.csv      
  inflating: train/793915326.csv     
  inflating: train/794676693.csv     
  inflating: train/794889827.csv     
  inflating: train/795059117.csv     
  inflating: train/795317395.csv     
  inflating: train/795941707.csv     
  inflating: train/795998482.csv     
  inflating: train/796309524.csv     
  inflating: train/796943128.csv     
  inflating: train/797097935.csv     
  inflating: train/797240914.csv     
  inflating: train/797838685.csv     
  inflating: train/7

In [ ]:
file_list_test = []
file_list_train = []
for dirname, _, filenames in os.walk('/content/test'):
    for filename in filenames:
        file_list_test.append(os.path.join(dirname, filename))


for dirname, _, filenames in os.walk('/content/train'):
    for filename in filenames:
        file_list_train.append(os.path.join(dirname, filename))

In [ ]:
time_file = pd.read_csv("/content/drive/MyDrive/train.csv")

# Сборка датасета

In [ ]:
dataMask = ["sum", "mean", "std", "max", "min", "skew", "quantile99", "quantile90", "quantile70",  "quantile20", "quantile10"]
metaDataDict = {}
for column in pd.read_csv(file_list_train[1]).keys():
    for mask in dataMask:
        metaDataDict[f"{column}_{mask}"] = []

In [ ]:
for file in tqdm.tqdm(time_file["segment_id"]):
    data = pd.read_csv(f"/content/train/{file}.csv")
    for column in data.keys():
        if np.isnan(data[column]).any():
            metaDataDict[f"{column}_sum"].append(0)
            metaDataDict[f"{column}_mean"].append(0)
            metaDataDict[f"{column}_std"].append(0)
            metaDataDict[f"{column}_max"].append(0)
            metaDataDict[f"{column}_min"].append(0)
            metaDataDict[f"{column}_skew"].append(0)
            metaDataDict[f"{column}_quantile99"].append(0)
            metaDataDict[f"{column}_quantile90"].append(0)
            metaDataDict[f"{column}_quantile70"].append(0)
            metaDataDict[f"{column}_quantile20"].append(0)
            metaDataDict[f"{column}_quantile10"].append(0)
        else:
            metaDataDict[f"{column}_sum"].append(data[column].sum()) #сумма
            metaDataDict[f"{column}_mean"].append(data[column].mean()) # среднее
            metaDataDict[f"{column}_std"].append(data[column].std()) # стандартное отклонение
            metaDataDict[f"{column}_max"].append(data[column].max()) # максимум
            metaDataDict[f"{column}_min"].append(data[column].min()) # минимум
            metaDataDict[f"{column}_skew"].append(data[column].skew()) # коэффициент асимметрии
            metaDataDict[f"{column}_quantile99"].append(np.quantile(data[column], 0.99)) #99% значений находятся ниже этого значения.
            metaDataDict[f"{column}_quantile90"].append(np.quantile(data[column], 0.90))
            metaDataDict[f"{column}_quantile70"].append(np.quantile(data[column], 0.70))
            metaDataDict[f"{column}_quantile20"].append(np.quantile(data[column], 0.20))
            metaDataDict[f"{column}_quantile10"].append(np.quantile(data[column], 0.10))

100%|██████████| 4431/4431 [13:31<00:00,  5.46it/s]


In [ ]:
for key in metaDataDict.keys():
    time_file[key] = metaDataDict[key]

<ipython-input-9-f2d79d01b0e8>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  time_file[key] = metaDataDict[key]
<ipython-input-9-f2d79d01b0e8>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  time_file[key] = metaDataDict[key]
<ipython-input-9-f2d79d01b0e8>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.c

In [ ]:
time_file = time_file.drop("segment_id", axis=1)

In [ ]:
gc.collect()

0

In [ ]:
X = time_file.drop("time_to_eruption", axis=1).copy()

# Нормализация

In [ ]:

scaler_standard = StandardScaler()
X_standardized = scaler_standard.fit_transform(X)

df_standardized = pd.DataFrame(X_standardized, columns=X.columns)

In [ ]:
x = df_standardized
y = time_file["time_to_eruption"]

xTrain, xTest, yTrain, yTest = train_test_split(x, y, train_size=0.8)

# Mодели

ExtraTreesRegressor - best

In [ ]:
model_ETR = ExtraTreesRegressor(max_depth=30)
model_ETR.fit(xTrain, yTrain)
yPred = model_ETR.predict(xTest)


In [ ]:
print(mean_absolute_error(yTest, yPred))
r2_score(yTest, yPred)

3552394.9464472923


0.8303540067771725

linear regression

In [ ]:
model_lin = LinearRegression()

model_lin.fit(xTrain, yTrain)

yPred = model_lin.predict(xTest)

In [ ]:
print(mean_absolute_error(yTest, yPred))
r2_score(yTest, yPred)

10116649.647805689


0.09855019065178894

ElasticNet


In [ ]:
model_EN = ElasticNetCV()
model_EN.fit(xTrain, yTrain)
yPred = model_EN.predict(xTest)

In [ ]:
print(mean_absolute_error(yTest, yPred))
r2_score(yTest, yPred)

11167383.88712964


0.0006136380873780212

Gradient boosting

In [ ]:
model_grad = ensemble.GradientBoostingRegressor()

param_grid = {
    "n_estimators": [100, 500, 1000, 1500, 2000],
    "max_depth": [3, 5, 8, 10],
    "min_samples_split": [2, 5, 10],
    "learning_rate": [0.01, 0.05, 0.1],
    "loss": ["squared_error", "absolute_error"],
}

grid_search = GridSearchCV(estimator=model_grad, param_grid=param_grid,
                           scoring='neg_mean_absolute_error', cv=5,
                           verbose=1, n_jobs=-1)

grid_search.fit(xTrain, yTrain)

best_params = grid_search.best_params_
print("Лучшие параметры:", best_params)


Fitting 5 folds for each of 360 candidates, totalling 1800 fits


In [ ]:
params = {
    "n_estimators": 1500,
    "max_depth": 8,
    "min_samples_split": 5,
    "learning_rate": 0.01,
    "loss": "squared_error",
}

model_grad = ensemble.GradientBoostingRegressor(**params)
model_grad.fit(xTrain, yTrain)
yPred = model_grad.predict(xTest)

In [ ]:
print(mean_absolute_error(yTest, yPred))
r2_score(yTest, yPred)

4227427.080323326


0.7839088072446607

# Стандартизация

In [ ]:
scaler_minmax = MinMaxScaler()
X_normalized = scaler_minmax.fit_transform(X)

df_normalized = pd.DataFrame(X, columns=X.columns)

In [ ]:
x = df_normalized
y = time_file["time_to_eruption"]

xTrain, xTest, yTrain, yTest = train_test_split(x, y, train_size=0.8)

# Mодели

ExtraTreesRegressor - best

In [ ]:
model_ETR = ExtraTreesRegressor(max_depth=30)
model_ETR.fit(xTrain, yTrain)
yPred = model_ETR.predict(xTest)


In [ ]:
print(mean_absolute_error(yTest, yPred))
r2_score(yTest, yPred)

3395061.76038064


0.8481710642913486

linear regression

In [ ]:
model_lin = LinearRegression()

model_lin.fit(xTrain, yTrain)

yPred = model_lin.predict(xTest)

In [ ]:
print(mean_absolute_error(yTest, yPred))
r2_score(yTest, yPred)

10084238.8943805


0.16693352540466866

ElasticNet


In [ ]:
model_EN = ElasticNetCV()
model_EN.fit(xTrain, yTrain)
yPred = model_EN.predict(xTest)

In [ ]:
print(mean_absolute_error(yTest, yPred))
r2_score(yTest, yPred)

10956747.681188125


0.07070892432780573

Gradient boosting

In [ ]:
model_grad = ensemble.GradientBoostingRegressor()

param_grid = {
    "n_estimators": [100, 500, 1000, 1500, 2000],
    "max_depth": [3, 5, 8, 10],
    "min_samples_split": [2, 5, 10],
    "learning_rate": [0.01, 0.05, 0.1],
    "loss": ["squared_error", "absolute_error"],
}

grid_search = GridSearchCV(estimator=model_grad, param_grid=param_grid,
                           scoring='neg_mean_absolute_error', cv=5,
                           verbose=1, n_jobs=-1)

grid_search.fit(xTrain, yTrain)

best_params = grid_search.best_params_
print("Лучшие параметры:", best_params)


Fitting 5 folds for each of 360 candidates, totalling 1800 fits


In [ ]:
params = {
    "n_estimators": 1500,
    "max_depth": 8,
    "min_samples_split": 5,
    "learning_rate": 0.01,
    "loss": "squared_error",
}

model_grad = ensemble.GradientBoostingRegressor(**params)
model_grad.fit(xTrain, yTrain)
yPred = model_grad.predict(xTest)

In [ ]:
print(mean_absolute_error(yTest, yPred))
r2_score(yTest, yPred)

3919496.1404889943


0.8106496175513229

# Нормализация и стандартизация

In [ ]:
scaler_standard = StandardScaler()
X_standardized = scaler_standard.fit_transform(X)

df_standardized = pd.DataFrame(X_standardized, columns=X.columns)

In [ ]:
scaler_minmax = MinMaxScaler()
X_normalized = scaler_minmax.fit_transform(X_standardized)

df_normalized = pd.DataFrame(X_normalized, columns=X.columns)

In [ ]:
x = df_normalized
y = time_file["time_to_eruption"]

xTrain, xTest, yTrain, yTest = train_test_split(x, y, train_size=0.8)

# Mодели

ExtraTreesRegressor - best

In [ ]:
model_ETR = ExtraTreesRegressor(max_depth=30)
model_ETR.fit(xTrain, yTrain)
yPred = model_ETR.predict(xTest)


In [ ]:
print(mean_absolute_error(yTest, yPred))
r2_score(yTest, yPred)

3589497.679980493


0.8376022105716987

linear regression

In [ ]:
model_lin = LinearRegression()

model_lin.fit(xTrain, yTrain)

yPred = model_lin.predict(xTest)

In [ ]:
print(mean_absolute_error(yTest, yPred))
r2_score(yTest, yPred)

10634113.657745471


-0.2986063537105992

ElasticNet


In [ ]:
model_EN = ElasticNetCV()
model_EN.fit(xTrain, yTrain)
yPred = model_EN.predict(xTest)

In [ ]:
print(mean_absolute_error(yTest, yPred))
r2_score(yTest, yPred)

11689830.921053557


-0.0012148961608584585

Gradient boosting

In [ ]:
model_grad = ensemble.GradientBoostingRegressor()

param_grid = {
    "n_estimators": [100, 500, 1000, 1500, 2000],
    "max_depth": [3, 5, 8, 10],
    "min_samples_split": [2, 5, 10],
    "learning_rate": [0.01, 0.05, 0.1],
    "loss": ["squared_error", "absolute_error"],
}

grid_search = GridSearchCV(estimator=model_grad, param_grid=param_grid,
                           scoring='neg_mean_absolute_error', cv=5,
                           verbose=1, n_jobs=-1)

grid_search.fit(xTrain, yTrain)

best_params = grid_search.best_params_
print("Лучшие параметры:", best_params)


Fitting 5 folds for each of 360 candidates, totalling 1800 fits


In [ ]:
params = {
    "n_estimators": 1500,
    "max_depth": 8,
    "min_samples_split": 5,
    "learning_rate": 0.01,
    "loss": "squared_error",
}

model_grad = ensemble.GradientBoostingRegressor(**params)
model_grad.fit(xTrain, yTrain)
yPred = model_grad.predict(xTest)

In [ ]:
print(mean_absolute_error(yTest, yPred))
r2_score(yTest, yPred)

4386543.925355009


0.7691662244416495